In [6]:
%load_ext dotenv
%dotenv

import os
from openai import AzureOpenAI
from azure.identity import ClientSecretCredential, get_bearer_token_provider
# Azure OpenAI Service Integration with LangChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [17]:
# 環境変数から認証情報を自動認識
# LangChain の中の AzureChatOpenAI インスタンス
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2025-01-01-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=6
)

In [3]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
list_instruction = CommaSeparatedListOutputParser().get_format_instructions()

In [4]:
print("List Instruction:", list_instruction)

List Instruction: Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [10]:
chat_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Could you suggest some names for a {pet}?\n"+list_instruction),
])

In [11]:
chat_value = chat_template.invoke({'pet':'dog'})

In [19]:
res = llm.invoke(chat_value)

In [20]:
list_output_parser = CommaSeparatedListOutputParser()
list_output_parser.invoke(res)

['Fluffy',
 'Bella',
 'Max',
 'Luna',
 'Charlie',
 'Daisy',
 'Milo',
 'Coco',
 'Rocky',
 'Oliver',
 'Ruby',
 'Teddy',
 'Hazel',
 'Finn',
 'Willow']

LangChain らしく Chain を使って書く

In [56]:
# pipe を使って invoke() を連鎖できる
chain = chat_template | llm | list_output_parser
chain.invoke({'pet': 'cat'})

['Of course! Could you']

ちなみに、LangChain の AzureChatOpenAI にサービスプリンシパル認証を適用するには？

In [60]:
from azure.identity import ClientSecretCredential, get_bearer_token_provider
from langchain_openai.chat_models import AzureChatOpenAI

credential = ClientSecretCredential(
    tenant_id=os.environ["tenant_id"],
    client_id=os.environ["client_id"],
    client_secret=os.environ["client_secret"]
)
token_provider = get_bearer_token_provider(
    credential,
    "https://cognitiveservices.azure.com/.default"
)
endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_version = "2025-01-01-preview"
model_deploy_name = "gpt-4o"
llm = AzureChatOpenAI(
    azure_deployment=model_deploy_name,
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
    temperature=0,
    max_tokens=None,
)

In [30]:
list_output_parser.invoke(llm.invoke(chat_value))

['Fluffy', 'Bella', '']

## Batch()
- invoke() を parallel で実行する

In [31]:
chat_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Could you suggest some names for a {pet} which is a {breed}?\n"+list_instruction),
])

In [64]:
chain = chat_template | llm
chain.invoke({'pet': 'dog', 'breed': 'pomeranian'})

AIMessage(content='Of course! Could you please specify the type of pet (e.g., dog, cat, etc.) and the breed?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 62, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_3eed281ddb', 'id': 'chatcmpl-C5LYAZES9qCKSfamnQiPvBr3E1XyZ', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_result

In [65]:
chain.batch([
    {'pet': 'dog', 'breed': 'pomeranian'},
    {'pet': 'cat', 'breed': 'siamese'}
])

[AIMessage(content='Of course! Could you please specify the type of pet (e.g., dog, cat, etc.) and the breed?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 62, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_3eed281ddb', 'id': 'chatcmpl-C5LYBRuNMUCgpZANPTGrtOkhK61Os', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_resul

In [66]:
res = chain.stream({'pet': 'dog', 'breed': 'pomeranian'})


In [68]:
next(res)

AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run--8f706274-23cf-413b-82b6-c9ed93e9fc1f')

In [69]:
for i in res:
    print(i.content, end='')

Of course! Could you please specify the type of pet (e.g., dog, cat, etc.) and the breed?